# Point Cloud Processing with Open3D

The field of 3D understanding has been attracting increasing attention in recent times, significantly propelled by (AR and Spatial Computing technology, backed by major companies like Apple and Meta, with recent the launch of Apple's Vision Pro). At the heart of this fascinating field lies 3D computer vision, a specialized branch of computer vision that focuses on understanding and processing three-dimensional visual data from the real world.

Applications range from navigation systems in self-driving cars and operational algorithms in autonomous robots to immersive virtual and augmented reality experiences. Processing and interpreting three-dimensional information is key to these technologies' development.

This article introduces 3D point cloud processing using Open3D library, an open-source library designed to empower developers and researchers with a comprehensive set of tools for 3D data processing. All these concepts will be explored through practical Python examples, providing a solid foundation for further exploration and application of 3D data processing skills.

## Initial setup

The Open3D official Python wheels offer support for Jupyter web visualizer, which requires building the Open3D Python package from source with the cmake option `-DBUILD_JUPYTER_EXTENSION` for enhanced functionality in Jupyter notebooks. To facilitate executing code in a Jupyter Notebook as if it were a script—particularly for visualization tasks that necessitate graphical interaction, such as those common with Open3D—a couple of methods are advisable.

A practical approach involves leveraging Jupyter Notebook's command-line interface (CLI) or an external environment to circumvent kernel blocking during visualization. Specifically, utilizing the `%%writefile` magic function allows for the saving of code into a `.py` file. This file can then be executed from a separate cell or through a terminal with the command `!python <script_name>.py`, enabling efficient management and execution of visualization tasks within the Jupyter ecosystem.


In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
import open3d as o3d

# Setting up paths for code and data
CODE_FOLDER = Path("../src").resolve()
CODE_FOLDER.mkdir(parents=True, exist_ok=True)

# Adding custom folders to the system path for easy import
sys.path.extend([str(CODE_FOLDER)])

## Visualizing a Point Cloud

The core of this tutorial focuses on loading and visualizing a point cloud using Open3D. The process begins with loading a .ply point cloud file, a popular format for storing 3D data. The read_point_cloud method is used for this purpose, which automatically decodes the file based on its extension.

In [2]:
%%writefile {CODE_FOLDER}/point_clouds_visualization.py

import numpy as np
import open3d as o3d

# Loading and visualizing a PLY point cloud
print("Loading a PLY point cloud, printing, and rendering...")
ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud(ply_point_cloud.path)

# Printing point cloud information and points array
print(pcd)
print(np.asarray(pcd.points))

# Setting visualization parameters
view_params = {
    "zoom": 0.3412,
    "front": [0.4257, -0.2125, -0.8795],
    "lookat": [2.6172, 2.0475, 1.532],
    "up": [-0.0694, -0.9768, 0.2024],
}

# Rendering the point cloud
o3d.visualization.draw_geometries([pcd], **view_params)



Overwriting /Users/carlos/Projects/3d_machine_learning/src/point_clouds_visualization.py


In [5]:
!python {CODE_FOLDER}/point_clouds_visualization.py

Loading a PLY point cloud, printing, and rendering...
PointCloud with 196133 points.
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]


After loading the point cloud, we can inspect it by printing the `pcd` object and the array of points it contains. This provides a glimpse into the structure and data contained within the point cloud. The visualization is handled by the `draw_geometries` function.

This powerful function renders the point cloud in a window, allowing users to interact with the 3D data. These parameters help in adjusting the camera's perspective, offering a detailed view of the data structure post-downsampling.



## Voxel Downsampling

This process applies a voxel grid to uniformly downsample an input point cloud. It simplifies point cloud data for computational efficiency in further processing steps. The method operates by grouping points into voxels and then averaging the points in each voxel to a single location.

This reduces the number of points, maintaining the general shape and features of the original point cloud.

In [6]:
%%writefile {CODE_FOLDER}/voxel_downsampling.py

import numpy as np
import open3d as o3d

print("Downsampling the point cloud with a voxel size of 0.05")

# Loading and visualizing a PLY point cloud
print("Loading a PLY point cloud, printing, and rendering...")
ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud(ply_point_cloud.path)

# Applying voxel downsampling
downpcd = pcd.voxel_down_sample(voxel_size=0.05)

# Setting visualization parameters for the downsampled point cloud
downsample_view_params = {
    "zoom": 0.3412,
    "front": [0.4257, -0.2125, -0.8795],
    "lookat": [2.6172, 2.0475, 1.532],
    "up": [-0.0694, -0.9768, 0.2024],
}

# Rendering the downsampled point cloud
o3d.visualization.draw_geometries([downpcd], **downsample_view_params)


Overwriting /Users/carlos/Projects/3d_machine_learning/src/voxel_downsampling.py


In [7]:
!python {CODE_FOLDER}/voxel_downsampling.py

Downsampling the point cloud with a voxel size of 0.05
Loading a PLY point cloud, printing, and rendering...


In the voxel downsampling code snippet, the process begins with specifying a voxel size of 0.05, which determines the resolution of the downsampling grid. The `voxel_down_sample` function is called on the original point cloud `pcd`, reducing its density by averaging the points within each voxel to a single point. This results in `downpcd`, a downsampled version of the original point cloud.

The `o3d.visualization.draw_geometries` function is then used to visualize `downpcd`. Parameters like `zoom`, `front`, `lookat`, and `up` are set to configure the viewing angle and position, providing a clear visualization of the downsampled point cloud.

## Vertex Normal Estimation

Vertex normal estimation calculates normals for each point in the cloud, crucial for many 3D processing tasks. Viewing normals can be enabled by pressing N, and their length adjusted with - and +. This function computes normals by identifying adjacent points and using covariance analysis to find the principal axis. A KDTreeSearchParamHybrid instance, specifying a search radius and maximum nearest neighbors, controls the detail of the normal estimation to balance between accuracy and computational load.

In [8]:
%%writefile {CODE_FOLDER}/vertex_normal_estimation.py

import numpy as np
import open3d as o3d

# Loading and visualizing a PLY point cloud
print("Loading a PLY point cloud, printing, and rendering...")
ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud(ply_point_cloud.path)

# Applying voxel downsampling
downpcd = pcd.voxel_down_sample(voxel_size=0.05)

print("Recomputing normals for the downsampled point cloud")

# Estimating normals for the downsampled point cloud
downpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)

# Visualization parameters for displaying normals
normals_view_params = {
    "zoom": 0.3412,
    "front": [0.4257, -0.2125, -0.8795],
    "lookat": [2.6172, 2.0475, 1.532],
    "up": [-0.0694, -0.9768, 0.2024],
    "point_show_normal": True,
}

# Rendering the downsampled point cloud with normals
o3d.visualization.draw_geometries([downpcd], **normals_view_params)


Overwriting /Users/carlos/Projects/3d_machine_learning/src/vertex_normal_estimation.py


In [9]:
!python {CODE_FOLDER}/vertex_normal_estimation.py

Loading a PLY point cloud, printing, and rendering...
Recomputing normals for the downsampled point cloud


In the vertex normal estimation code, the operation begins by calling `estimate_normals` on the downsampled point cloud `downpcd`. This function calculates normals for each point, which are essential for many 3D processing tasks, such as rendering, simulation, and further geometric analysis. The `search_param` argument specifies how the normals are computed, using a KD-tree to efficiently find nearby points.

Here, `o3d.geometry.KDTreeSearchParamHybrid` is configured with a `radius` of 0.1 and a `max_nn` (maximum nearest neighbors) of 30. This configuration balances the precision of normal estimation with computational efficiency by limiting the search to a radius of 10cm and considering up to 30 neighbors for each point.

Following the computation of normals, the `o3d.visualization.draw_geometries` function visualizes the downsampled point cloud with normals enabled.